# 정수 인코딩( Integer Encoding )

#### 컴퓨터는 텍스트보다는 숫자를 더 잘 처리할 수 있다. 자연어 처리에서는 텍스트를 숫자로 바꾸는 여러가지 기법이 있다. 이러한 기법을 적용하는 첫 단계로 각 단어를 고유한 정수에 mapping 시키는 전처리 작업이 필요할 때가 있다.

## 1. 정수 인코딩( Integer Encoding )

#### 단어에 정수를 부여하는 방법 중 하나로 단어를 빈도수 순으로 정렬한 단어 집합( vocabulary )을 만들고, 빈도수가 높은 순서대로 차례로 낮은 숫자부터 정수를 부여하는 방법이 있다.

### 1) dictionary 사용하기

In [2]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [12]:
text = "A barber is a person. a barber is good person. a barber is huge person. he knew A secret! The Secret He Kept is huge secret. Huge secret. His barber kept his word. a barber kept his word. His barber kept his secret. But keeping and keeping such a huge secret to himself was driving the barber crazy. the barber went up a huge mountain."

In [13]:
# 문장 토큰화
text = sent_tokenize( text )
print( text )

['A barber is a person.', 'a barber is good person.', 'a barber is huge person.', 'he knew A secret!', 'The Secret He Kept is huge secret.', 'Huge secret.', 'His barber kept his word.', 'a barber kept his word.', 'His barber kept his secret.', 'But keeping and keeping such a huge secret to himself was driving the barber crazy.', 'the barber went up a huge mountain.']


In [14]:
# 정제와 단어 토큰화
vocab = {}
sentences = []
stop_words = set( stopwords.words( 'english' ) )

for i in text:
    sentence = word_tokenize( i ) # 단어 토큰화
    result = []
    
    for word in sentence:
        word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄인다.
        if word not in stop_words: # 단어 토큰화 된 결과에 대해서 불용어를 제거
            if len( word ) > 2: # 단어 길이가 2이하인 경우에 대하여 추가로 단어 제거
                result.append( word )
                if word not in vocab:
                    vocab[ word ] = 0
                vocab[ word ] += 1
    sentences.append( result )

print( sentences )

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


#### 텍스트를 숫자로 바꾸는 단계라는 것은 본격적으로 자연어 처리 작업에 들어간다는 의미이므로, 단어가 텍스트일 때만 할 수 이쓴ㄴ 최대한의 전처리를 끝내놓아야 한다. 위 코드에서 동일한 단어가 대문자로 표기되었다는 이유로 서로 다른 단어로 카운트되는 일이 없도록 모든 단어를 소문자로 바꾸었다. 그리고 자연어 처리에서 크게 의미를 갖지 못하는 불용어와 길이가 짧은 단어를 제거하는 방법을 사용하였다.

#### vocab에는 중복을 제거한 단어와 각 단어에 대한 빈도수가 기록되어져 있다.

In [15]:
print( vocab )

{'barber': 8, 'person': 3, 'good': 1, 'huge': 5, 'knew': 1, 'secret': 6, 'kept': 4, 'word': 2, 'keeping': 2, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1}


#### vocab에는 단어를 key로, 단어에 대한 빈도수가 value로 저장되어져 있다. vocab에 단어를 입력하면 빈도수가 리턴된다.

In [16]:
print( vocab[ "barber" ] )

8


#### 빈도수가 높은 순으로 정렬

In [17]:
vocab_sorted = sorted( vocab.items(), key = lambda x : x[  1 ], reverse = True )
print( vocab_sorted )

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3), ('word', 2), ('keeping', 2), ('good', 1), ('knew', 1), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)]


#### 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여

In [18]:
word_to_index = {}
i = 0
for ( word, frequency ) in vocab_sorted:
    if frequency > 1 : # 빈도수가 적은 단어는 제외한다.
        i += 1
        word_to_index[ word ] = i 
print( word_to_index )

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7}


#### 1의 인덱스를 가진 단어가 가장 빈도수가 높은 단어가 된다. 그리고 이러한 작업을 수행하는 동시에 각 단어의 빈도수를 알 경우만 할 수 있는 전처리인 빈도수가 적은 단어를 제외시키는 작업을 한다.

#### 등장 빈도가 낮은 단어는 자연어 처리에서 의미를 가지지 않을 가능성이 높기 때문이다. 

#### 자연어 처리를 하다보면, 텍스트 데이터에 있는 단어를 모두 사용하기 보다는 빈도수가 가장 높은 n개의 단어만 사용하고 싶은 경우가 많다. 위의 코드에서는 빈도수가 높은 순으로 낮은 정수가 부여되어져 있으므로 빈도수 상위 n개의 단어만 사용하고 싶다고 하며 vocab에서 정수값이 1부터 n개까지인 단어들만 사용하면 된다.

In [19]:
vacab_size = 5
words_frequency = [ w for w, c in word_to_index.items() if c >= vacab_size + 1 ] # 인덱스가 5 초과인 단어 제거
for w in words_frequency:
    del word_to_index[ w ] # 해당 단어에 대한 인덱스 정보를 삭제
print( word_to_index ) # 빈도수가 높은 상위 5개의 단어만 저장

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


#### word_to_index를 사용하여 단어 토큰화가 된 상태로 저장된 sentences에 있는 각 단어를 정수로 바꾸는 작업을 한다.
#### 예를 들어 sentences에서 [ 'barber', 'person' ]을 [ 1, 5 ]로 인코딩한다.
#### [ 'barber', 'good', 'person' ] 에는 더 이상 word_to_index에는 존재하지 않는 단어인 'good'이라는 단어가 있다.
#### 이처럼 훈련 데이터 또는 테스트 데이터에 대해서 단어 집합에 존재하지 않는 단어들을 Out-Of-Vocabulary( 단어 집합에 없는 단어 )의 약자로 OOV라 한다.
#### word_to_index에 OOV란 단어를 새롭게 추가하고, 단어 집합에 없는 단어들은 OOV의 인덱스로 인코딩한다.

In [21]:
word_to_index[ 'OOV' ] = len( word_to_index ) + 1

#### word_to_index를 사용하여 sentences의 모든 단어들을 mapping 되는 정수로 인코딩한다.

In [22]:
encoded = []
for s in sentences:
    temp = []
    for w in s:
        try:
            temp.append( word_to_index[ w ] )
        except KeyError:
            temp.append( word_to_index[ 'OOV' ] )
    encoded.append( temp )
print( encoded )

[[1, 5], [1, 6, 5], [1, 3, 5], [6, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [6, 6, 3, 2, 6, 1, 6], [1, 6, 3, 6]]


#### 파이썬의 dictionary 자료형으로 정수 인코디을 진행하였는데, 이보다는 좀 더 쉽게 하기 위해서 Counter, FreqDist, enumerate, Keras tokenizer를 사용하는 것을 귄장

### 2) Counter 사용하기

In [23]:
from collections import Counter

In [24]:
print( sentences )

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


#### 단어 집합( vocabulary )를 만들기 위해 sentences에서 문장의 경계인 '[,]'를 제거하고 단어들을 하나의 리스트로 만든다.

In [25]:
words = sum( sentences, [] ) # words = np.hstack( sentences )로도 수행 가능
print( words )

['barber', 'person', 'barber', 'good', 'person', 'barber', 'huge', 'person', 'knew', 'secret', 'secret', 'kept', 'huge', 'secret', 'huge', 'secret', 'barber', 'kept', 'word', 'barber', 'kept', 'word', 'barber', 'kept', 'secret', 'keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy', 'barber', 'went', 'huge', 'mountain']


#### 파이썬의 Counter()의 입력으로 사용하면 중복을 제거하고 빈도수를 기록한다.

In [26]:
vocab = Counter( words ) # 파이썬의 Counter 모듈을 사용하면 단어의 모든 빈도를 쉽게 계산할 수 있다.
print( vocab )

Counter({'barber': 8, 'secret': 6, 'huge': 5, 'kept': 4, 'person': 3, 'word': 2, 'keeping': 2, 'good': 1, 'knew': 1, 'driving': 1, 'crazy': 1, 'went': 1, 'mountain': 1})


In [27]:
print( vocab[ 'barber' ] )

8


#### most_common()는 상위 빈도수를 가진 주어진 수의 단어만을 리턴한다. 이를 사용하여 등장 빈도수가 높은 단어들을 원하는 개수만큼만 얻을 수 있다.

In [28]:
vocab_size = 5
vocab = vocab.most_common( vocab_size ) # 등장 빈도수가 높은 상위 4개의 단어만 저장
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

#### 높은 빈도수를 가진 단어일수록 낮은 정수 인덱스를 부여한다.

In [29]:
word_to_index = {}
i = 0
for ( word, frequency ) in vocab:
    i += 1
    word_to_index[ word ] = i
print( word_to_index )

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


### 3) NLTK의 FreqDist 사용하기

#### NLTK에서는 빈도수 계산 도구인 FreqDist()를 지원한다. Counter()와 같은 방법으로 사용할 수 있다.

In [30]:
from nltk import FreqDist
import numpy as np

In [32]:
# np.hstack으로 문자 구분을 제거하여 입력으로 사용. ex) [ 'barber', 'person', 'barber', 'good', ... ]
vocab = FreqDist( np.hstack( sentences ) )
print( vocab )

<FreqDist with 13 samples and 36 outcomes>


In [33]:
print( vocab[ 'barber' ] )

8


In [34]:
vocat_size = 5
vocab = vocab.most_common( vocab_size )
vocab

[('barber', 8), ('secret', 6), ('huge', 5), ('kept', 4), ('person', 3)]

In [35]:
word_to_index = { word[ 0 ] : index + 1 for index, word in enumerate( vocab ) }
print( word_to_index )

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}


### 4) enumerate 이해하기

#### enumerate()는 순서가 있는 자료형( list, set, tuple, dictionary, string )을 입력으로 받아 인덱스를 순차적으로 함께 리턴한다는 특징이 있다.

In [36]:
test = [ 'a', 'b', 'c', 'd', 'e' ]
for index, value in enumerate( test ): # 입력의 순서대로 0부터 인덱스를 부여함
    print( "value : {}, index : {}".format( value, index ) )

value : a, index : 0
value : b, index : 1
value : c, index : 2
value : d, index : 3
value : e, index : 4


## 2. Keras의 텍스트 전처리

#### Keras는 기본적인 전처리를 위한 도구들을 제공한다. 때로는 정수 인코딩을 위해서 Keras의 전처리 도구인 Tokenizer를 사용하기도 한다.

In [37]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [38]:
print( sentences )

[['barber', 'person'], ['barber', 'good', 'person'], ['barber', 'huge', 'person'], ['knew', 'secret'], ['secret', 'kept', 'huge', 'secret'], ['huge', 'secret'], ['barber', 'kept', 'word'], ['barber', 'kept', 'word'], ['barber', 'kept', 'secret'], ['keeping', 'keeping', 'huge', 'secret', 'driving', 'barber', 'crazy'], ['barber', 'went', 'huge', 'mountain']]


In [39]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts( sentences ) # fit_on_texts()안에 corpus를 입력으로 하면 빈도수를 기준으로 단어 집합을 생성한다.

#### fit_on_texts()는 입력한 텍스트로부터 단어 빈도수가 높은 순으로 낮은 정수 인덱스를 부여한다. 각 단어에 인덱스가 어떻게 부여되었는지를 보려면, word_index를 사용한다.

In [40]:
print( tokenizer.word_index )

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5, 'word': 6, 'keeping': 7, 'good': 8, 'knew': 9, 'driving': 10, 'crazy': 11, 'went': 12, 'mountain': 13}


#### 각 단어가 카운트를 수행하였을 때 몇 개였는지를 보고자 한다면 word_counts를 사용한다.

In [41]:
print( tokenizer.word_counts )

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


#### texts_to_sequences()는 입력으로 들어온 corpus에 대해서 각 단어를 이미 정해진 인덱스로 변환한다.

In [42]:
print( tokenizer.texts_to_sequences( sentences ) )

[[1, 5], [1, 8, 5], [1, 3, 5], [9, 2], [2, 4, 3, 2], [3, 2], [1, 4, 6], [1, 4, 6], [1, 4, 2], [7, 7, 3, 2, 10, 1, 11], [1, 12, 3, 13]]


#### 앞서 빈도수가 가장 높은 단어 n개만을 사용하기 위해서 most_common()을 사용했으나, Keras Tokenizer에서는 tokenizer = Tokenizer( num_words = 숫자 )와 같은 방법으로 빈도수가 높은 상위 몇 개의  단어만 사용하겠다고 지정할 수 있다.

In [43]:
vocab_size = 5
tokenizer = Tokenizer( num_words = vocab_size + 1 ) # 상위 5개 단어만 사용
tokenizer.fit_on_texts( sentences )

#### num_words에서 +1을 더해서 값을 넣어주는 이유는 num_words는 0부터 카운트한다. 만약 5를 넣으면 0 ~ 4번 단어 보존을 의미하게 되므로 1번 단어부터 4번 단어만 남게됩니다. 그렇기 때문에 1 ~ 5번 단어까지 사용하고 싶다면 num_words에 숫자 5를 넣어주는 것이 아니라 5 + 1인 값을 넣어 주어야 한다.

#### 실질적으로 숫자 0에 지정된 단어가 존재하지 않는데도 Keras Tokenizer가 숫자 0까지 단어 집합의 크기로 산정하는 이유는 자연어 처리에서 padding이라는 작업 때문이다. Keras Tokenizer를 사용할 때는 숫자 0도 단어 집합의 크기로 고려해야 한다.

In [44]:
print( tokenizer.word_counts )

OrderedDict([('barber', 8), ('person', 3), ('good', 1), ('huge', 5), ('knew', 1), ('secret', 6), ('kept', 4), ('word', 2), ('keeping', 2), ('driving', 1), ('crazy', 1), ('went', 1), ('mountain', 1)])


In [45]:
print( tokenizer.texts_to_sequences( sentences ) )

[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


#### corpus에 대해서 각 단어를 이미 정해진 인덱스로 변환하는데, 상위 5개의 단어만을 사용하겠다고 지정하였으므로 1번 단어부터 5번 단어까지만 보존되고 나머지 단어들은 제거된 것을 볼 수 있다.

#### word_index와 word_counts에서도 지정된 num_words만큼의 단어만 남기고 싶다면 아래 코드도 하나의 방법이다.

In [46]:
tokenizer = Tokenizer() # num_words를 지정하지 않은 상태
tokenizer.fit_on_texts( sentences )

In [47]:
vocab_size = 5
words_frequency = [ w for w, c in tokenizer.word_index.items() if c >= vocab_size + 1 ] # 인덱스가 5 초과인 단어 제거
for w in words_frequency:
    del tokenizer.word_index[ w ] # 해당 단어에 대한 인덱스 정보를 삭제
    del tokenizer.word_counts[ w ] # 해당 단어에 대한 카운트 정보를 삭제
print( tokenizer.word_index )
print( tokenizer.word_counts )
print( tokenizer.texts_to_sequences( sentences ) )

{'barber': 1, 'secret': 2, 'huge': 3, 'kept': 4, 'person': 5}
OrderedDict([('barber', 8), ('person', 3), ('huge', 5), ('secret', 6), ('kept', 4)])
[[1, 5], [1, 5], [1, 3, 5], [2], [2, 4, 3, 2], [3, 2], [1, 4], [1, 4], [1, 4, 2], [3, 2, 1], [1, 3]]


#### Keras Tokenizer는 기본적으로 단어 집합에 없는 단어인 OOV에 대해서는 단어를 정수로 바꾸는 과정에서 아예 단어를 제거한다는 특징이 있다. 단어 집합에 없는 단어들은 OOV로 간주하여 보존하고 싶다면 Tokenizer의 인자 oov_token을 사용한다.

In [48]:
vocab_size = 5
tokenizer = Tokenizer( num_words = vocab_size + 2, oov_token = 'OOV' ) 
# 빈도수 상위 5개 단어만 사용, 숫자 0과 OOV를 고려해서 단어 집합의 크기는 +2
tokenizer.fit_on_texts( sentences )

#### 만약 oov_token을 사용하기로 했다면 Keras Tokenizer는 기본적으로 'OOV'의 인덱스를 1로 한다.

In [49]:
print( '단어 OOV의 인덱스 : {}'.format( tokenizer.word_index[ 'OOV' ] ) )

단어 OOV의 인덱스 : 1


#### 이제 corpus에 대해서 정수 인코딩을 진행한다.

In [50]:
print( tokenizer.texts_to_sequences( sentences ) )

[[2, 6], [2, 1, 6], [2, 4, 6], [1, 3], [3, 5, 4, 3], [4, 3], [2, 5, 1], [2, 5, 1], [2, 5, 3], [1, 1, 4, 3, 1, 2, 1], [2, 1, 4, 1]]


#### 빈두수 상위 5개의 단어는 2 ~ 6까지의 인덱스를 가졌으며, 그 외 단어 집합에 없는 'good'과 같은 단어들은 전부 'OOV'의 인덱스인 1로 인코딩되었다.